In [1]:
# Recommender System case 1

# The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/).    
# It is similar to using the iris or MNIST data set for other algorithms. 
# file upload location
# '/FileStore/tables/movielens_ratings.csv'

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [5]:
# ### Alternating Least Squares (ALS) matrix factorization

# ALS attempts to estimate the ratings matrix R as the product of two lower-rank matrices, X and Y, i.e. X * Yt = R. Typically these approximations are called ‘factor’ matrices. The general approach is iterative. During each iteration, one of the factor matrices is held constant, while the other is solved for using least squares. The newly-solved factor matrix is then held constant while solving for the other factor matrix.

# This is a blocked implementation of the ALS factorization algorithm that groups the two sets of factors (referred to as “users” and “products”) into blocks and reduces communication by only sending one copy of each user vector to each product block on each iteration, and only for the product blocks that need that user’s feature vector. This is achieved by pre-computing some information about the ratings matrix to determine the “out-links” of each user (which blocks of products it will contribute to) and “in-link” information for each product (which of the feature vectors it receives from each user block it will depend on). This allows us to send only an array of feature vectors between each user block and product block, and have the product block find the users’ ratings and update the products based on these messages.

# For implicit preference data, the algorithm used is based on “Collaborative Filtering for Implicit Feedback Datasets”,, adapted for the blocked approach used here.

# Essentially instead of finding the low-rank approximations to the rating matrix R, this finds the approximations for a preference matrix P where the elements of P are 1 if r > 0 and 0 if r <= 0. The ratings then act as ‘confidence’ values related to strength of indicated user preferences rather than explicit ratings given to items.

In [6]:
data = spark.read.csv('/FileStore/tables/movielens_ratings.csv',inferSchema=True,header=True)

In [7]:
data.head(5)

Out[11]: [Row(movieId=2, rating=3.0, userId=0),
 Row(movieId=3, rating=1.0, userId=0),
 Row(movieId=5, rating=2.0, userId=0),
 Row(movieId=9, rating=4.0, userId=0),
 Row(movieId=11, rating=1.0, userId=0)]

In [8]:
data.describe().show()

+-------+------------------+------------------+------------------+
summary| movieId| rating| userId|
+-------+------------------+------------------+------------------+
 count| 1501| 1501| 1501|
 mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
 stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
 min| 0| 1.0| 0|
 max| 99| 5.0| 29|
+-------+------------------+------------------+------------------+

In [9]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [10]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

In [11]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [12]:
predictions.show()

+-------+------+------+-----------+
movieId|rating|userId| prediction|
+-------+------+------+-----------+
 31| 1.0| 24| 1.7349851|
 31| 1.0| 29| 0.86477846|
 85| 1.0| 28|-0.44283506|
 85| 1.0| 26| 3.3411899|
 85| 2.0| 20| 3.8999066|
 85| 1.0| 5| -1.5466185|
 85| 1.0| 29| 0.3997574|
 65| 1.0| 16| 1.4032907|
 65| 2.0| 3| 3.4671044|
 65| 2.0| 5| 2.988379|
 53| 1.0| 12| 3.4875195|
 53| 3.0| 13| 1.5718421|
 53| 1.0| 23| 0.12725769|
 78| 1.0| 27| 0.5842633|
 78| 1.0| 17| 1.2707244|
 78| 1.0| 8| 0.54235613|
 34| 1.0| 4| -2.533222|
 34| 3.0| 25| 0.18056911|
 81| 1.0| 22|-0.53513485|
 81| 1.0| 21| 1.6268234|
+-------+------+------+-----------+
only showing top 20 rows

In [13]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.857159385507095

In [14]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [15]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
movieId|userId|
+-------+------+
 0| 11|
 16| 11|
 19| 11|
 27| 11|
 35| 11|
 36| 11|
 39| 11|
 47| 11|
 66| 11|
 70| 11|
 88| 11|
+-------+------+

In [16]:
reccomendations = model.transform(single_user)

In [17]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
movieId|userId| prediction|
+-------+------+-----------+
 88| 11| 4.0848866|
 36| 11| 1.5214694|
 39| 11| 0.97764695|
 35| 11| 0.6643993|
 16| 11| 0.6604533|
 0| 11| 0.5620753|
 66| 11|-0.44784492|
 27| 11|-0.52199304|
 19| 11|-0.66633266|
 70| 11| -1.1178101|
 47| 11| -3.7614796|
+-------+------+-----------+